In [ ]:
results_axial_pca = []
results_axial_ols = []
results_midpoints_pca = []
results_midpoints_ols = []

# Define a minimum number of points required to fit a line
MIN_POINTS_FOR_FIT = 2

# Iterate over each frame index present in the original df_dlc
for frame_idx in df_dlc.index:
    frame_results = {} # To store results for the current frame

    for set_name, coords_dfs in bodypart_coordinate_sets.items():
        current_x_df = coords_dfs['x']
        current_y_df = coords_dfs['y']

        # Initialize parameters to NaN for the current frame and set
        pca_dx, pca_dy, pca_mean_x, pca_mean_y, pca_explained_variance = np.nan, np.nan, np.nan, np.nan, np.nan
        ols_slope, ols_intercept, ols_r_value, ols_p_value = np.nan, np.nan, np.nan, np.nan

        if frame_idx in current_x_df.index and frame_idx in current_y_df.index:
            x_coords_frame = current_x_df.loc[frame_idx]
            y_coords_frame = current_y_df.loc[frame_idx]

            # Combine into a DataFrame and drop NaNs
            points_df = pd.DataFrame({'x': x_coords_frame, 'y': y_coords_frame}).dropna()

            if len(points_df) >= MIN_POINTS_FOR_FIT:
                # --- PCA ---
                try:
                    pca = PCA(n_components=1)
                    pca.fit(points_df[['x', 'y']].values)
                    pca_dx, pca_dy = pca.components_[0]
                    pca_mean_x, pca_mean_y = pca.mean_
                    pca_explained_variance = pca.explained_variance_[0] if pca.explained_variance_ is not None else np.nan
                except Exception as e:
                    pass 

                # --- OLS (y = slope*x + intercept) ---
                try:
                    if points_df['x'].nunique() > 1: 
                        slope, intercept, r_val, p_val, _ = linregress(points_df['x'], points_df['y'])
                        ols_slope, ols_intercept, ols_r_value, ols_p_value = slope, intercept, r_val, p_val
                    elif len(points_df) >= MIN_POINTS_FOR_FIT : 
                        pass 
                except ValueError as ve: 
                    pass
                except Exception as e:
                    pass

        # Store results based on the set_name
        if set_name == "axial_only":
            results_axial_pca.append({
                'frame': frame_idx, 'pca_dx': pca_dx, 'pca_dy': pca_dy, 
                'pca_mean_x': pca_mean_x, 'pca_mean_y': pca_mean_y,
                'pca_explained_variance': pca_explained_variance
            })
            results_axial_ols.append({
                'frame': frame_idx, 'ols_slope': ols_slope, 'ols_intercept': ols_intercept,
                'ols_r_value': ols_r_value, 'ols_p_value': ols_p_value
            })
        elif set_name == "axial_plus_midpoints":
            results_midpoints_pca.append({
                'frame': frame_idx, 'pca_dx': pca_dx, 'pca_dy': pca_dy, 
                'pca_mean_x': pca_mean_x, 'pca_mean_y': pca_mean_y,
                'pca_explained_variance': pca_explained_variance
            })
            results_midpoints_ols.append({
                'frame': frame_idx, 'ols_slope': ols_slope, 'ols_intercept': ols_intercept,
                'ols_r_value': ols_r_value, 'ols_p_value': ols_p_value
            })

# Convert lists of dictionaries to DataFrames
df_axial_pca_raw = pd.DataFrame(results_axial_pca).set_index('frame')
df_axial_ols_raw = pd.DataFrame(results_axial_ols).set_index('frame')
df_midpoints_pca_raw = pd.DataFrame(results_midpoints_pca).set_index('frame')
df_midpoints_ols_raw = pd.DataFrame(results_midpoints_ols).set_index('frame')

print("\n--- Calculation Complete (with extended metrics) ---")
print("\nShape of df_axial_pca_raw:", df_axial_pca_raw.shape)
print(df_axial_pca_raw.head())
print("\nShape of df_axial_ols_raw:", df_axial_ols_raw.shape)
print(df_axial_ols_raw.head())
print("\nShape of df_midpoints_pca_raw:", df_midpoints_pca_raw.shape)
print(df_midpoints_pca_raw.head())
print("\nShape of df_midpoints_ols_raw:", df_midpoints_ols_raw.shape)
print(df_midpoints_ols_raw.head())

print("\n--- NaN counts per method (with extended metrics) ---")
print(f"Axial PCA NaNs:\n{df_axial_pca_raw.isna().sum()}")
print(f"\nAxial OLS NaNs:\n{df_axial_ols_raw.isna().sum()}")
print(f"\nMidpoints PCA NaNs:\n{df_midpoints_pca_raw.isna().sum()}")
print(f"\nMidpoints OLS NaNs:\n{df_midpoints_ols_raw.isna().sum()}")

In [ ]:
required_globals = [
    'df_axial_pca_raw', 'df_axial_ols_raw', 
    'df_midpoints_pca_raw', 'df_midpoints_ols_raw',
    'filtered_x_coords', 'filtered_y_coords', 'frame_rate',
    'output_dir_path', 'base_output_name', 'save_plots', 'df_dlc'
]
for var_name in required_globals:
    if var_name not in globals():
        raise NameError(f"Required global variable '{var_name}' not found. Please ensure previous cells have run.")

print("\n--- Calculating Angles of Body Axis vs. Positive Y-axis ---")

# --- Orientation Setup ---
front_bp_name = 'neck'
back_bp_name = 'mid_backend'
can_orient_globally = True # Renamed for clarity

if not all(bp in filtered_x_coords.columns for bp in [front_bp_name, back_bp_name]) or \
   not all(bp in filtered_y_coords.columns for bp in [front_bp_name, back_bp_name]):
    print(f"Warning: Orientation bodyparts ('{front_bp_name}', '{back_bp_name}') not fully available in filtered_x_coords/filtered_y_coords.")
    print("Angles will be calculated based on raw PCA/OLS vectors without anatomical orientation.")
    can_orient_globally = False
    # Create empty series if orientation parts are missing, so .get() doesn't fail later
    front_bp_x_series = pd.Series(dtype=float, index=df_dlc.index)
    front_bp_y_series = pd.Series(dtype=float, index=df_dlc.index)
    back_bp_x_series = pd.Series(dtype=float, index=df_dlc.index)
    back_bp_y_series = pd.Series(dtype=float, index=df_dlc.index)
else:
    front_bp_x_series = filtered_x_coords[front_bp_name]
    front_bp_y_series = filtered_y_coords[front_bp_name]
    back_bp_x_series = filtered_x_coords[back_bp_name]
    back_bp_y_series = filtered_y_coords[back_bp_name]
    print(f"Using '{front_bp_name}' (front) and '{back_bp_name}' (back) for anatomical orientation of vectors.")

# --- Calculate Percentage of Frames Where Anatomical Orientation Was Applied ---
frames_oriented_successfully = 0
total_frames_for_orientation_check = len(df_dlc.index)
percentage_oriented = 0.0

if can_orient_globally and total_frames_for_orientation_check > 0:
    for frame_idx in df_dlc.index:
        f_x = front_bp_x_series.get(frame_idx, np.nan)
        f_y = front_bp_y_series.get(frame_idx, np.nan)
        b_x = back_bp_x_series.get(frame_idx, np.nan)
        b_y = back_bp_y_series.get(frame_idx, np.nan)
        
        if not (pd.isna(f_x) or pd.isna(f_y) or pd.isna(b_x) or pd.isna(b_y) or (f_x == b_x and f_y == b_y)):
            frames_oriented_successfully += 1
    
    if total_frames_for_orientation_check > 0:
        percentage_oriented = (frames_oriented_successfully / total_frames_for_orientation_check) * 100

# --- Helper Function for Angle Calculation (PCA) ---
def calculate_pca_angle_vs_y(df_pca_raw, method_name):
    angles = []
    for frame_idx, row in df_pca_raw.iterrows():
        dx, dy = row['pca_dx'], row['pca_dy']
        if pd.isna(dx) or pd.isna(dy):
            angles.append(np.nan)
            continue

        oriented_dx, oriented_dy = dx, dy
        if can_orient_globally: # Use the global flag
            f_x, f_y = front_bp_x_series.get(frame_idx, np.nan), front_bp_y_series.get(frame_idx, np.nan)
            b_x, b_y = back_bp_x_series.get(frame_idx, np.nan), back_bp_y_series.get(frame_idx, np.nan)

            if not (pd.isna(f_x) or pd.isna(f_y) or pd.isna(b_x) or pd.isna(b_y) or (f_x == b_x and f_y == b_y)):
                anat_dx, anat_dy = f_x - b_x, f_y - b_y
                if (dx * anat_dx + dy * anat_dy) < 0: 
                    oriented_dx, oriented_dy = -dx, -dy
        
        angles.append(np.degrees(np.arctan2(oriented_dx, oriented_dy)))
    df_pca_raw[f'angle_y_deg_{method_name}'] = angles
    print(f"Calculated angles for PCA ({method_name}).")

# --- Helper Function for Angle Calculation (OLS) ---
def calculate_ols_angle_vs_y(df_ols_raw, method_name):
    angles = []
    for frame_idx, row in df_ols_raw.iterrows():
        slope = row['ols_slope']
        if pd.isna(slope):
            angles.append(np.nan)
            continue
        
        base_dx, base_dy = 1.0, slope 
        oriented_dx, oriented_dy = base_dx, base_dy

        if can_orient_globally: # Use the global flag
            f_x, f_y = front_bp_x_series.get(frame_idx, np.nan), front_bp_y_series.get(frame_idx, np.nan)
            b_x, b_y = back_bp_x_series.get(frame_idx, np.nan), back_bp_y_series.get(frame_idx, np.nan)

            if not (pd.isna(f_x) or pd.isna(f_y) or pd.isna(b_x) or pd.isna(b_y) or (f_x == b_x and f_y == b_y)):
                anat_dx, anat_dy = f_x - b_x, f_y - b_y
                if (base_dx * anat_dx + base_dy * anat_dy) < 0:
                    oriented_dx, oriented_dy = -base_dx, -base_dy
            
        angles.append(np.degrees(np.arctan2(oriented_dx, oriented_dy)))
    df_ols_raw[f'angle_y_deg_{method_name}'] = angles
    print(f"Calculated angles for OLS ({method_name}).")

# --- Calculate Angles for all 4 methods ---
calculate_pca_angle_vs_y(df_axial_pca_raw, 'axial_pca')
calculate_ols_angle_vs_y(df_axial_ols_raw, 'axial_ols')
calculate_pca_angle_vs_y(df_midpoints_pca_raw, 'midpoints_pca')
calculate_ols_angle_vs_y(df_midpoints_ols_raw, 'midpoints_ols')

# --- Prepare Time Axis for Plotting ---
# Assuming all _raw DataFrames share the same index as df_dlc
time_axis = df_dlc.index / frame_rate

# --- Plotting ---
plot_configs = [
    {'df': df_axial_pca_raw, 'col': 'angle_y_deg_axial_pca', 'label': 'Axial PCA', 'title': 'Angle: Axial PCA vs. +Y Axis'},
    {'df': df_axial_ols_raw, 'col': 'angle_y_deg_axial_ols', 'label': 'Axial OLS', 'title': 'Angle: Axial OLS vs. +Y Axis'},
    {'df': df_midpoints_pca_raw, 'col': 'angle_y_deg_midpoints_pca', 'label': 'Midpoints PCA', 'title': 'Angle: Midpoints PCA vs. +Y Axis'},
    {'df': df_midpoints_ols_raw, 'col': 'angle_y_deg_midpoints_ols', 'label': 'Midpoints OLS', 'title': 'Angle: Midpoints OLS vs. +Y Axis'},
]

# Individual Plots
for config in plot_configs:
    if config['col'] in config['df'].columns:
        plt.figure(figsize=(12, 4))
        plt.plot(time_axis, config['df'][config['col']], label=config['label'], linewidth=0.8)
        plt.title(config['title'])
        plt.xlabel("Time (s)")
        plt.ylabel("Angle (degrees)")
        plt.ylim(-190, 190)
        plt.yticks(np.arange(-180, 181, 45))
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        if save_plots:
            plot_filename = os.path.join(output_dir_path, f"{base_output_name}_{config['label'].replace(' ', '_').lower()}_angle_vs_y.png")
            try:
                plt.savefig(plot_filename, dpi=150)
                print(f"Saved plot: {plot_filename}")
            except Exception as e:
                print(f"Error saving plot {plot_filename}: {e}")
        plt.show()
    else:
        print(f"Column {config['col']} not found in DataFrame for {config['label']}. Skipping individual plot.")

# Combined Plot
plt.figure(figsize=(15, 6))
any_plotted = False
for config in plot_configs:
    if config['col'] in config['df'].columns:
        plt.plot(time_axis, config['df'][config['col']], label=config['label'], linewidth=1)
        any_plotted = True

if any_plotted:
    plt.title("Comparison: Body Axis Angle vs. Positive Y-axis")
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (degrees)")
    plt.ylim(-190, 190)
    plt.yticks(np.arange(-180, 181, 45))
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    if save_plots:
        plot_filename = os.path.join(output_dir_path, f"{base_output_name}_combined_angles_vs_y.png")
        try:
            plt.savefig(plot_filename, dpi=150)
            print(f"Saved combined plot: {plot_filename}")
        except Exception as e:
            print(f"Error saving combined plot {plot_filename}: {e}")
    plt.show()
else:
    print("No data available to plot for the combined angle comparison.")

# --- Orientation Summary ---
print("\n--- Orientation Summary ---")
if not can_orient_globally:
    print(f"Anatomical orientation was not attempted because required bodyparts ('{front_bp_name}', '{back_bp_name}') were not consistently available in the filtered coordinate data.")
    print("Percentage of frames where anatomical orientation was applied (common to all methods): 0.00%")
else:
    print(f"Anatomical orientation was attempted using '{front_bp_name}' (front) and '{back_bp_name}' (back) for all methods.")
    print(f"Percentage of frames where these specific bodyparts were valid and distinct, allowing anatomical orientation to be applied: {percentage_oriented:.2f}%")
    if percentage_oriented < 100 and total_frames_for_orientation_check > 0 : # Avoid printing if 0% or 100%
        print(f"For the remaining {100.0 - percentage_oriented:.2f}% of frames (where orientation was attempted but reference points were NaN or identical), raw PCA/OLS vectors were used for angle calculation.")
    elif total_frames_for_orientation_check == 0:
        print("No frames available to assess orientation percentage.")


print("\n--- Angle plotting complete. ---")

In [ ]:
from scipy.signal import savgol_filter
required_globals = [
    'df_midpoints_pca_raw', 'frame_rate', 
    'output_dir_path', 'base_output_name', 'save_plots', 'df_dlc'
]
for var_name in required_globals:
    if var_name not in globals():
        raise NameError(f"Required global variable '{var_name}' not found. Please ensure previous cells have run.")

if 'angle_y_deg_midpoints_pca' not in df_midpoints_pca_raw.columns:
    raise KeyError("'angle_y_deg_midpoints_pca' column not found in df_midpoints_pca_raw. Please run the angle calculation cell.")

print("\n--- Applying Smoothing to Midpoints PCA Angles ---")

# --- Data and Parameters ---
angle_data_series = df_midpoints_pca_raw['angle_y_deg_midpoints_pca'].copy()
time_axis = df_dlc.index / frame_rate # Assuming df_midpoints_pca_raw shares index with df_dlc

smoothing_window_frames = 15
savgol_poly_order = 2

# Ensure window is odd for Savitzky-Golay, if not, make it odd (e.g., add 1)
if smoothing_window_frames % 2 == 0:
    smoothing_window_frames +=1
    print(f"Adjusted Savitzky-Golay window to be odd: {smoothing_window_frames} frames")

rolling_mean_smoothed_angle = angle_data_series.rolling(window=smoothing_window_frames, center=True, min_periods=1).mean()
print(f"Applied rolling mean smoothing with window: {smoothing_window_frames} frames.")

# --- Savitzky-Golay Smoothing ---
# Since it's confirmed that angle_data_series (from angle_y_deg_midpoints_pca) has no NaNs,
# interpolation is not needed. We still check for data length.

savgol_smoothed_angle = pd.Series(np.nan, index=angle_data_series.index) # Initialize with NaNs

if angle_data_series.empty:
    print("Skipping Savitzky-Golay: 'angle_data_series' is empty.")
elif angle_data_series.isna().any(): # Safeguard: This should be false based on prior NaN check.
    print("Skipping Savitzky-Golay: Unexpected NaNs found in 'angle_data_series'.")
    print("Please re-run the NaN check cell. If NaNs persist, data cleaning might be needed.")
elif len(angle_data_series) < smoothing_window_frames:
    print(f"Skipping Savitzky-Golay: Data length ({len(angle_data_series)}) is less than window size ({smoothing_window_frames}).")
else:
    # Data is not empty, expected to have no NaNs, and is long enough.
    savgol_smoothed_values = savgol_filter(angle_data_series, # Directly use the original series
                                           window_length=smoothing_window_frames, 
                                           polyorder=savgol_poly_order,
                                           mode='mirror') # 'mirror' handles edges better
    savgol_smoothed_angle.loc[angle_data_series.index] = savgol_smoothed_values
    # No need to re-apply original NaNs as angle_data_series is assumed to have no NaNs.
    print(f"Applied Savitzky-Golay smoothing with window: {smoothing_window_frames} frames, polyorder: {savgol_poly_order}.")

# --- Plotting ---
# Plot 1: Original Data
plt.figure(figsize=(15, 5))
plt.plot(time_axis, angle_data_series, label='Original Midpoints PCA Angle', color='grey', alpha=0.7, linewidth=1)
plt.title('Midpoints PCA Angle: Original Data')
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees vs +Y axis)")
plt.ylim(-190, 190)
plt.yticks(np.arange(-180, 181, 45))
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
if save_plots:
    plot_filename_original = os.path.join(output_dir_path, f"{base_output_name}_midpoints_pca_angle_original.png")
    try:
        plt.savefig(plot_filename_original, dpi=150)
        print(f"Saved original angle plot: {plot_filename_original}")
    except Exception as e:
        print(f"Error saving original angle plot {plot_filename_original}: {e}")
plt.show()

# Plot 2: Rolling Mean Smoothed Data
plt.figure(figsize=(15, 5))
plt.plot(time_axis, rolling_mean_smoothed_angle, label=f'Rolling Mean (w={smoothing_window_frames})', color='blue', linewidth=1.5)
plt.title(f'Midpoints PCA Angle: Rolling Mean Smoothed (Window: {smoothing_window_frames} frames)')
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees vs +Y axis)")
plt.ylim(-190, 190)
plt.yticks(np.arange(-180, 181, 45))
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
if save_plots:
    plot_filename_rolling = os.path.join(output_dir_path, f"{base_output_name}_midpoints_pca_angle_rolling_mean.png")
    try:
        plt.savefig(plot_filename_rolling, dpi=150)
        print(f"Saved rolling mean smoothed angle plot: {plot_filename_rolling}")
    except Exception as e:
        print(f"Error saving rolling mean smoothed angle plot {plot_filename_rolling}: {e}")
plt.show()

# Plot 3: Savitzky-Golay Smoothed Data
if not savgol_smoothed_angle.isna().all(): # Only plot if Savitzky-Golay was applied and resulted in some non-NaN data
    plt.figure(figsize=(15, 5))
    plt.plot(time_axis, savgol_smoothed_angle, label=f'Savitzky-Golay (w={smoothing_window_frames}, p={savgol_poly_order})', color='red', linewidth=1.5)
    plt.title(f'Midpoints PCA Angle: Savitzky-Golay Smoothed (Window: {smoothing_window_frames}, Polyorder: {savgol_poly_order})')
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (degrees vs +Y axis)")
    plt.ylim(-190, 190)
    plt.yticks(np.arange(-180, 181, 45))
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    if save_plots:
        plot_filename_savgol = os.path.join(output_dir_path, f"{base_output_name}_midpoints_pca_angle_savgol.png")
        try:
            plt.savefig(plot_filename_savgol, dpi=150)
            print(f"Saved Savitzky-Golay smoothed angle plot: {plot_filename_savgol}")
        except Exception as e:
            print(f"Error saving Savitzky-Golay smoothed angle plot {plot_filename_savgol}: {e}")
    plt.show()
else:
    print("Savitzky-Golay smoothed data was all NaN or not computed (e.g. data too short), skipping its individual plot.")

# Combined Plot
plt.figure(figsize=(15, 7)) # Keep combined plot for comparison
plt.plot(time_axis, angle_data_series, label='Original Midpoints PCA Angle', color='grey', alpha=0.5, linewidth=1)
plt.plot(time_axis, rolling_mean_smoothed_angle, label=f'Rolling Mean (w={smoothing_window_frames})', color='blue', linewidth=1.5)
if not savgol_smoothed_angle.isna().all(): # Only plot if Savitzky-Golay was applied
    plt.plot(time_axis, savgol_smoothed_angle, label=f'Savitzky-Golay (w={smoothing_window_frames}, p={savgol_poly_order})', color='red', linewidth=1.5)

plt.title('Midpoints PCA Angle: Original and Smoothed')
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees vs +Y axis)")
plt.ylim(-190, 190)
plt.yticks(np.arange(-180, 181, 45))
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

if save_plots:
    plot_filename = os.path.join(output_dir_path, f"{base_output_name}_midpoints_pca_angle_smoothed_comparison.png") # Slightly different name for clarity
    try:
        plt.savefig(plot_filename, dpi=150)
        print(f"Saved combined smoothed angle plot: {plot_filename}")
    except Exception as e:
        print(f"Error saving combined smoothed angle plot {plot_filename}: {e}")
plt.show()

print("\n--- Smoothing and plotting complete. ---")

In [ ]:
print("\n--- Quantitative Comparison of Line Fitting Methods ---")

# --- PCA Metrics ---
print("\n-- PCA --")
if 'df_axial_pca_raw' in globals() and 'pca_explained_variance' in df_axial_pca_raw.columns:
    mean_expl_var_axial = df_axial_pca_raw['pca_explained_variance'].mean()
    std_expl_var_axial = df_axial_pca_raw['pca_explained_variance'].std()
    print(f"Axial Only - PCA Explained Variance: Mean={mean_expl_var_axial:.2f}, StdDev={std_expl_var_axial:.2f}")

    # PCA Angle Stability
    angle_pca_axial = np.arctan2(df_axial_pca_raw['pca_dy'], df_axial_pca_raw['pca_dx'])
    angle_diff_pca_axial = angle_pca_axial.diff()
    # Correct for angle wrapping (results in radians, between -pi and pi)
    angle_diff_pca_axial = (angle_diff_pca_axial + np.pi) % (2 * np.pi) - np.pi
    std_angle_diff_pca_axial = angle_diff_pca_axial.std() # radians
    print(f"Axial Only - PCA Angle Stability (StdDev of frame-to-frame change): {np.degrees(std_angle_diff_pca_axial):.2f} degrees")

if 'df_midpoints_pca_raw' in globals() and 'pca_explained_variance' in df_midpoints_pca_raw.columns:
    mean_expl_var_midpoints = df_midpoints_pca_raw['pca_explained_variance'].mean()
    std_expl_var_midpoints = df_midpoints_pca_raw['pca_explained_variance'].std()
    print(f"Midpoints - PCA Explained Variance: Mean={mean_expl_var_midpoints:.2f}, StdDev={std_expl_var_midpoints:.2f}")

    angle_pca_midpoints = np.arctan2(df_midpoints_pca_raw['pca_dy'], df_midpoints_pca_raw['pca_dx'])
    angle_diff_pca_midpoints = angle_pca_midpoints.diff()
    angle_diff_pca_midpoints = (angle_diff_pca_midpoints + np.pi) % (2 * np.pi) - np.pi
    std_angle_diff_pca_midpoints = angle_diff_pca_midpoints.std() # radians
    print(f"Midpoints - PCA Angle Stability (StdDev of frame-to-frame change): {np.degrees(std_angle_diff_pca_midpoints):.2f} degrees")

# --- OLS Metrics ---
print("\n-- OLS --")
SIGNIFICANCE_THRESHOLD = 0.05

if 'df_axial_ols_raw' in globals() and 'ols_r_value' in df_axial_ols_raw.columns:
    df_axial_ols_raw['ols_r_squared'] = df_axial_ols_raw['ols_r_value']**2
    mean_r_sq_axial = df_axial_ols_raw['ols_r_squared'].mean()
    std_r_sq_axial = df_axial_ols_raw['ols_r_squared'].std()
    print(f"Axial Only - OLS R-squared: Mean={mean_r_sq_axial:.3f}, StdDev={std_r_sq_axial:.3f}")
    
    if 'ols_p_value' in df_axial_ols_raw.columns:
        significant_fits_axial = (df_axial_ols_raw['ols_p_value'] < SIGNIFICANCE_THRESHOLD).sum()
        percent_significant_axial = (significant_fits_axial / len(df_axial_ols_raw.dropna(subset=['ols_p_value']))) * 100
        print(f"Axial Only - OLS P-value < {SIGNIFICANCE_THRESHOLD}: {percent_significant_axial:.2f}% of frames")

    # OLS Slope Stability
    slope_diff_ols_axial = df_axial_ols_raw['ols_slope'].diff()
    std_slope_diff_ols_axial = slope_diff_ols_axial.std()
    print(f"Axial Only - OLS Slope Stability (StdDev of frame-to-frame change): {std_slope_diff_ols_axial:.3f}")


if 'df_midpoints_ols_raw' in globals() and 'ols_r_value' in df_midpoints_ols_raw.columns:
    df_midpoints_ols_raw['ols_r_squared'] = df_midpoints_ols_raw['ols_r_value']**2
    mean_r_sq_midpoints = df_midpoints_ols_raw['ols_r_squared'].mean()
    std_r_sq_midpoints = df_midpoints_ols_raw['ols_r_squared'].std()
    print(f"Midpoints - OLS R-squared: Mean={mean_r_sq_midpoints:.3f}, StdDev={std_r_sq_midpoints:.3f}")

    if 'ols_p_value' in df_midpoints_ols_raw.columns:
        significant_fits_midpoints = (df_midpoints_ols_raw['ols_p_value'] < SIGNIFICANCE_THRESHOLD).sum()
        percent_significant_midpoints = (significant_fits_midpoints / len(df_midpoints_ols_raw.dropna(subset=['ols_p_value']))) * 100
        print(f"Midpoints - OLS P-value < {SIGNIFICANCE_THRESHOLD}: {percent_significant_midpoints:.2f}% of frames")

    slope_diff_ols_midpoints = df_midpoints_ols_raw['ols_slope'].diff()
    std_slope_diff_ols_midpoints = slope_diff_ols_midpoints.std()
    print(f"Midpoints - OLS Slope Stability (StdDev of frame-to-frame change): {std_slope_diff_ols_midpoints:.3f}")

print("\n--- End of Quantitative Comparison ---")

In [ ]:
num_sample_frames = 5 # How many sample frames to plot
# Select some frames (e.g., start, middle, end of available frames)
# Ensure these frame indices exist in your df_dlc.index
total_frames = len(df_dlc.index)

# Ensure the raw PCA/OLS dataframes exist
required_dfs = ['df_axial_pca_raw', 'df_axial_ols_raw', 'df_midpoints_pca_raw', 'df_midpoints_ols_raw', 'bodypart_coordinate_sets', 'df_dlc']
for req_df_name in required_dfs:
    if req_df_name not in globals():
        raise NameError(f"Required DataFrame '{req_df_name}' not found. Please ensure previous cells have been run.")

if 'output_dir_path' not in globals() or 'base_output_name' not in globals():
    raise NameError("Variables 'output_dir_path' or 'base_output_name' not found. Ensure cell 'd3f3a5c2' has run.")

# Ensure save_plots is defined, defaulting to False if not (though it should be from cell 'd650d79e')
if 'save_plots' not in globals():
    print("Warning: 'save_plots' variable not found. Defaulting to False for this cell.")
    save_plots_this_cell = False
else:
    save_plots_this_cell = save_plots


if total_frames == 0:
    print("Error: df_dlc has no frames. Cannot plot sample frames.")
else:
    sample_frame_indices = np.linspace(0, total_frames - 1, num_sample_frames, dtype=int)
    sample_frames = [df_dlc.index[i] for i in sample_frame_indices]

    print(f"\n--- Plotting Body Part Sets with PCA and OLS Lines for Sample Frames: {sample_frames} ---")

    ref_point_front_name = 'neck'
    ref_point_back_name = 'mid_backend'

    for frame_idx in sample_frames:
        fig, axes = plt.subplots(1, 2, figsize=(16, 7)) # Increased figsize for better legend visibility
        fig.suptitle(f"Body Axis Estimation for Frame: {frame_idx}", fontsize=16)

        plot_idx = 0
        for set_name, coords_dfs in bodypart_coordinate_sets.items():
            ax = axes[plot_idx]
            current_x_df = coords_dfs['x']
            current_y_df = coords_dfs['y']

            ax.set_title(f"Point Set: '{set_name}'")
            ax.set_xlabel("X-coordinate")
            ax.set_ylabel("Y-coordinate")
            ax.invert_yaxis() 
            ax.set_aspect('equal', adjustable='box')
            ax.grid(True, linestyle='--', alpha=0.7)

            if frame_idx not in current_x_df.index or frame_idx not in current_y_df.index:
                ax.text(0.5, 0.5, "Frame data not in index", ha='center', va='center')
                plot_idx += 1
                continue

            x_coords_frame_series = current_x_df.loc[frame_idx].dropna()
            y_coords_frame_series = current_y_df.loc[frame_idx].dropna()
            
            common_index = x_coords_frame_series.index.intersection(y_coords_frame_series.index)
            x_coords_frame = x_coords_frame_series[common_index]
            y_coords_frame = y_coords_frame_series[common_index]

            if not x_coords_frame.empty:
                ax.scatter(x_coords_frame, y_coords_frame, label='Body Points in Set', alpha=0.6, s=50)

                # Highlight reference points
                if ref_point_front_name in x_coords_frame.index:
                    ax.scatter(x_coords_frame[ref_point_front_name], y_coords_frame[ref_point_front_name],
                               color='cyan', s=120, label=f'{ref_point_front_name} (Front Ref)', edgecolors='black', zorder=5, marker='^')
                if ref_point_back_name in x_coords_frame.index:
                    ax.scatter(x_coords_frame[ref_point_back_name], y_coords_frame[ref_point_back_name],
                               color='magenta', s=120, label=f'{ref_point_back_name} (Back Ref)', edgecolors='black', zorder=5, marker='v')

                # --- Plot PCA Line ---
                if set_name == "axial_only":
                    pca_params = df_axial_pca_raw.loc[frame_idx]
                else: # axial_plus_midpoints
                    pca_params = df_midpoints_pca_raw.loc[frame_idx]

                if not pca_params.isna().any():
                    pca_dx, pca_dy = pca_params['pca_dx'], pca_params['pca_dy']
                    pca_mean_x, pca_mean_y = pca_params['pca_mean_x'], pca_params['pca_mean_y']
                    
                    line_len = 75 
                    x_pca = np.array([pca_mean_x - line_len * pca_dx, pca_mean_x + line_len * pca_dx])
                    y_pca = np.array([pca_mean_y - line_len * pca_dy, pca_mean_y + line_len * pca_dy])
                    ax.plot(x_pca, y_pca, color='red', linestyle='-', linewidth=2, label=f'PCA Line', zorder=3) # Simplified label
                    ax.scatter(pca_mean_x, pca_mean_y, color='red', s=50, edgecolor='black', zorder=4, label=f'PCA Mean') # Simplified label


                # --- Plot OLS Line ---
                if set_name == "axial_only":
                    ols_params = df_axial_ols_raw.loc[frame_idx]
                else: # axial_plus_midpoints
                    ols_params = df_midpoints_ols_raw.loc[frame_idx]

                if not ols_params.isna().any():
                    ols_slope, ols_intercept = ols_params['ols_slope'], ols_params['ols_intercept']
                    
                    xlim_data = (x_coords_frame.min(), x_coords_frame.max())
                    current_ax_xlim = ax.get_xlim()

                    # Determine plot range for OLS line: prefer axis limits if set, else data limits
                    plot_x_min = min(xlim_data[0], current_ax_xlim[0]) if ax.lines or ax.collections else xlim_data[0]
                    plot_x_max = max(xlim_data[1], current_ax_xlim[1]) if ax.lines or ax.collections else xlim_data[1]
                    
                    if plot_x_min == plot_x_max : 
                        plot_x_min -= 10
                        plot_x_max += 10
                    
                    x_ols = np.array([plot_x_min, plot_x_max])
                    y_ols = ols_slope * x_ols + ols_intercept
                    ax.plot(x_ols, y_ols, color='green', linestyle='--', linewidth=2, label=f'OLS Line', zorder=2) # Simplified label

            else:
                ax.text(0.5, 0.5, "No valid points for this frame", ha='center', va='center')
            
            ax.legend(fontsize='small')
            plot_idx += 1
        
        plt.tight_layout(rect=[0, 0, 1, 0.95]) 

        if save_plots_this_cell:
            plot_filename = os.path.join(output_dir_path, f"{base_output_name}_axis_comparison_frame_{frame_idx}.png")
            try:
                plt.savefig(plot_filename, dpi=300)
                print(f"Saved axis comparison plot to: {plot_filename}")
            except Exception as e:
                print(f"Error saving axis comparison plot for frame {frame_idx}: {e}")
        
        plt.show()

if total_frames == 0:
     print("Plotting skipped as df_dlc is empty.")

In [ ]:
num_sample_frames = 5 # How many sample frames to plot
total_frames_in_df = len(df_dlc.index) # Use df_dlc to determine available frames

# Ensure the raw PCA/OLS dataframes exist
required_dfs = ['df_axial_pca_raw', 'df_axial_ols_raw', 'df_midpoints_pca_raw', 'df_midpoints_ols_raw', 'bodypart_coordinate_sets', 'df_dlc']
for req_df_name in required_dfs:
    if req_df_name not in globals():
        raise NameError(f"Required DataFrame '{req_df_name}' not found. Please ensure previous cells have been run.")

if 'output_dir_path' not in globals() or 'base_output_name' not in globals():
    raise NameError("Variables 'output_dir_path' or 'base_output_name' not found. Ensure cell 'd3f3a5c2' has run.")

if 'video_file_path' not in globals() or not os.path.exists(video_file_path):
    print("Warning: Video file path not found or video file does not exist. Skipping frame overlay plotting.")
    # Optionally, set a flag to skip this cell's plotting logic
    # skip_plotting_on_frames = True 
else:
    # skip_plotting_on_frames = False
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_file_path}")
        # skip_plotting_on_frames = True
    else:
        video_total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Video file opened successfully: {video_file_path} ({video_total_frames} frames)")


# Ensure save_plots is defined
save_plots_this_cell = globals().get('save_plots', False)
if 'save_plots' not in globals():
    print("Warning: 'save_plots' variable not found. Defaulting to False for saving plots from this cell.")


if total_frames_in_df == 0:
    print("Error: df_dlc has no frames. Cannot plot sample frames.")
# elif skip_plotting_on_frames:
#    print("Skipping plotting on frames due to video file issue.")
elif 'cap' not in locals() or not cap.isOpened():
    print("Skipping plotting on frames as video capture is not available.")
else:
    # Select frames that exist in both the DataFrame and the video
    # Ensure sample_frame_indices are within the bounds of video_total_frames
    valid_indices_for_video = [i for i in range(total_frames_in_df) if df_dlc.index[i] < video_total_frames]
    
    if not valid_indices_for_video:
        print("Error: No frames from df_dlc are within the video's frame range.")
    else:
        # Adjust num_sample_frames if fewer valid frames are available
        actual_num_samples = min(num_sample_frames, len(valid_indices_for_video))
        if actual_num_samples < num_sample_frames:
            print(f"Warning: Requested {num_sample_frames} samples, but only {actual_num_samples} are valid within video range.")

        if actual_num_samples > 0:
            sample_df_indices = np.linspace(0, len(valid_indices_for_video) - 1, actual_num_samples, dtype=int)
            # Get the actual frame numbers (as in df_dlc.index and for cap.set)
            sample_frames_to_plot = [df_dlc.index[valid_indices_for_video[i]] for i in sample_df_indices]
        else:
            sample_frames_to_plot = []

        print(f"\n--- Plotting Body Part Sets with PCA and OLS Lines on Video Frames for: {sample_frames_to_plot} ---")

        ref_point_front_name = 'neck'
        ref_point_back_name = 'mid_backend'
        
        # Define colors (BGR for OpenCV)
        color_body_points = (255, 150, 0)  # Light Blue
        color_ref_front = (0, 255, 255)   # Yellow
        color_ref_back = (255, 0, 255)    # Magenta
        color_pca_line = (0, 0, 255)      # Red
        color_pca_mean = (0, 0, 255)      # Red
        color_ols_line = (0, 255, 0)      # Green

        for frame_idx_to_plot in sample_frames_to_plot:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx_to_plot)
            ret, frame_bgr = cap.read()
            if not ret:
                print(f"Warning: Could not read frame {frame_idx_to_plot} from video.")
                continue

            fig, axes = plt.subplots(1, 2, figsize=(18, 7)) # Adjusted figsize
            fig.suptitle(f"Body Axis Estimation on Frame: {frame_idx_to_plot}", fontsize=16)

            plot_subplot_idx = 0
            for set_name, coords_dfs in bodypart_coordinate_sets.items():
                ax = axes[plot_subplot_idx]
                ax.set_title(f"Point Set: '{set_name}'")
                ax.axis('off') # Turn off axis numbers and ticks for image display

                frame_to_draw_on = frame_bgr.copy()

                current_x_df_set = coords_dfs['x']
                current_y_df_set = coords_dfs['y']

                if frame_idx_to_plot not in current_x_df_set.index or frame_idx_to_plot not in current_y_df_set.index:
                    # This case should ideally be handled by how sample_frames_to_plot are chosen
                    # but as a fallback, show the raw frame
                    frame_rgb_display = cv2.cvtColor(frame_to_draw_on, cv2.COLOR_BGR2RGB)
                    ax.imshow(frame_rgb_display)
                    ax.text(0.5, 0.5, "Point set data not in index for this frame", color='white',
                            ha='center', va='center', transform=ax.transAxes, backgroundcolor='black')
                    plot_subplot_idx += 1
                    continue

                x_coords_frame_series = current_x_df_set.loc[frame_idx_to_plot].dropna()
                y_coords_frame_series = current_y_df_set.loc[frame_idx_to_plot].dropna()
                
                common_index = x_coords_frame_series.index.intersection(y_coords_frame_series.index)
                x_coords_frame = x_coords_frame_series[common_index]
                y_coords_frame = y_coords_frame_series[common_index]

                # Draw body points
                for bp_name in common_index:
                    pt_x, pt_y = int(x_coords_frame[bp_name]), int(y_coords_frame[bp_name])
                    cv2.circle(frame_to_draw_on, (pt_x, pt_y), 5, color_body_points, -1)
                    if bp_name == ref_point_front_name:
                        cv2.circle(frame_to_draw_on, (pt_x, pt_y), 7, color_ref_front, -1)
                        cv2.circle(frame_to_draw_on, (pt_x, pt_y), 7, (0,0,0), 1) # Black outline
                    elif bp_name == ref_point_back_name:
                        cv2.circle(frame_to_draw_on, (pt_x, pt_y), 7, color_ref_back, -1)
                        cv2.circle(frame_to_draw_on, (pt_x, pt_y), 7, (0,0,0), 1) # Black outline


                # --- Plot PCA Line ---
                pca_params_df = df_axial_pca_raw if set_name == "axial_only" else df_midpoints_pca_raw
                if frame_idx_to_plot in pca_params_df.index:
                    pca_params = pca_params_df.loc[frame_idx_to_plot]
                    if not pca_params.isna().any():
                        pca_dx, pca_dy = pca_params['pca_dx'], pca_params['pca_dy']
                        pca_mean_x, pca_mean_y = pca_params['pca_mean_x'], pca_params['pca_mean_y']
                        
                        line_len = 75 
                        x1 = int(pca_mean_x - line_len * pca_dx)
                        y1 = int(pca_mean_y - line_len * pca_dy)
                        x2 = int(pca_mean_x + line_len * pca_dx)
                        y2 = int(pca_mean_y + line_len * pca_dy)
                        cv2.line(frame_to_draw_on, (x1, y1), (x2, y2), color_pca_line, 2)
                        cv2.circle(frame_to_draw_on, (int(pca_mean_x), int(pca_mean_y)), 6, color_pca_mean, -1)

                # --- Plot OLS Line ---
                ols_params_df = df_axial_ols_raw if set_name == "axial_only" else df_midpoints_ols_raw
                if frame_idx_to_plot in ols_params_df.index:
                    ols_params = ols_params_df.loc[frame_idx_to_plot]
                    if not ols_params.isna().any() and not x_coords_frame.empty: # Ensure points exist for OLS line
                        ols_slope, ols_intercept = ols_params['ols_slope'], ols_params['ols_intercept']
                        
                        x_min_ols_pts = x_coords_frame.min()
                        x_max_ols_pts = x_coords_frame.max()

                        if x_min_ols_pts == x_max_ols_pts: # Avoid division by zero or zero-length line if all x are same
                             # Draw a short vertical segment if slope is huge, or rely on PCA
                            if abs(ols_slope) > 1e3: # Heuristic for very steep slope
                                y_center = ols_slope * x_min_ols_pts + ols_intercept
                                cv2.line(frame_to_draw_on, (int(x_min_ols_pts), int(y_center - line_len/2)), 
                                         (int(x_min_ols_pts), int(y_center + line_len/2)), color_ols_line, 2)
                            # else: OLS line might not be meaningful here, could skip drawing
                        else:
                            y_start_ols = int(ols_slope * x_min_ols_pts + ols_intercept)
                            y_end_ols = int(ols_slope * x_max_ols_pts + ols_intercept)
                            cv2.line(frame_to_draw_on, (int(x_min_ols_pts), y_start_ols), (int(x_max_ols_pts), y_end_ols), color_ols_line, 2)
                
                frame_rgb_display = cv2.cvtColor(frame_to_draw_on, cv2.COLOR_BGR2RGB)
                ax.imshow(frame_rgb_display)
                plot_subplot_idx += 1
            
            plt.tight_layout(rect=[0, 0, 1, 0.95]) 

            if save_plots_this_cell:
                plot_filename = os.path.join(output_dir_path, f"{base_output_name}_axis_on_frame_{frame_idx_to_plot}.png")
                try:
                    plt.savefig(plot_filename, dpi=150) # Lower DPI for potentially large images
                    print(f"Saved axis-on-frame plot to: {plot_filename}")
                except Exception as e:
                    print(f"Error saving axis-on-frame plot for frame {frame_idx_to_plot}: {e}")
            
            plt.show()

    if 'cap' in locals() and cap.isOpened():
        cap.release()

if total_frames_in_df == 0:
     print("Plotting skipped as df_dlc is empty.")
elif 'cap' in locals() and not cap.isOpened() and os.path.exists(globals().get('video_file_path', '')):
    print("Plotting on frames skipped because video could not be opened.")